In [1]:
from dateutil import parser, tz
import pytz
from hashlib import sha256
import pandas as pd
import numpy as np

In [2]:
def unix_time(time: str, utc: bool=True):
    '''Accepts time as a string and returns unix time.'''
    t = parser.parse(time)
    if utc:
        t = t.replace(tzinfo=tz.gettz('UTC')) # if the time is in UTC, est the timezone

    return '{0:08x}'.format(int(t.timestamp()))

In [3]:
#TEST UNIC_TIME FUNCTION
genesis_time = '2009-01-03 18:15:05' # time is in UTC
#'0x495fab29'
timeout = unix_time(genesis_time)
print (timeout)

495fab29


In [4]:
def little_endian(hexa: str):

    '''Returns an integer corresponding to little endian representation'''
    
    return int.from_bytes(bytearray.fromhex(hexa.replace('0x', '')), byteorder='little')


In [5]:
hex(little_endian(unix_time(genesis_time)))

'0x29ab5f49'

In [6]:
def block_header_generator(version, perv_block_hash, merkle_root, time, bits, utc):
    '''Generates the block header.'''
    
    version_hex = '{0:08x}'.format(little_endian(version))
    perv_hash_hex = '{0:064x}'.format(little_endian(perv_block_hash))			
    merkle_hex = '{0:064x}'.format(little_endian(merkle_root))
    time_hex = '{0:08x}'.format(little_endian(unix_time(time, utc)))
    bits_hex = '{0:08x}'.format(little_endian('{0:08x}'.format(bits)))
    header = [version_hex, perv_hash_hex, merkle_hex, time_hex, bits_hex]

    return ''.join(header)	

In [7]:
#TEST BLOCK HEADER
b_version = '0x' + '1'.zfill(8) # = '0x00000001'
b_merkle = '4a5e1e4baab89f3a32518a88c31bc87f618f76673e2cc77ab2127b7afdeda33b'
b_perv_hash = '00'
b_time = '2009-01-03 18:15:05'
b_bits = 486604799

print

<function print>

In [8]:
correct_result = '0100000000000000000000000000000000000000000000000000000000000000000003\ba3edfd7a7b12b27ac72c3e67768f617fc81bc3888a51323a9fb8aa4b1e4a29ab5f49ffff001d'


In [9]:
correct_result == block_header_generator(b_version, b_perv_hash, b_merkle, b_time, b_bits, utc=True)

False

In [10]:
def target_cal(bits: int):

    '''Calculates the target value for comparing.'''
    
    h = '{0:08x}'.format(bits)
    return int(h[2:], 16) * (256 ** (int(h[:2], 16)-3))

In [11]:
def mine(b_version, b_perv_hash, b_merkle, b_time, b_bits, near_value, utc=True):
    '''performs the process of mining.'''
    
    nonce = near_value # for speeding up the process
    target = target_cal(b_bits)
    b = block_header_generator(b_version, b_perv_hash, b_merkle, b_time, b_bits, utc)
    print('Start Mining...')
    while True:
        nonce_little = '{0:08x}'.format(little_endian('{0:08x}'.format(nonce)))
        header = b + nonce_little
        first_hash = sha256(bytes.fromhex(header)) # nedss byte format
        second_hash = sha256 (first_hash.digest()).hexdigest()
        result = int.from_bytes(bytearray.fromhex(second_hash), 'little') # for comparing with the target
        if result < target:
            print('Block Mined ! ', 'Nonce : ',nonce, sep='\n')
            print('hash : ', '{0:064x}'.format(result), sep='\n')
            break
        nonce +=1

In [12]:
near_value = 2073236893
mine(b_version, b_perv_hash, b_merkle, b_time, b_bits, near_value)

Start Mining...
Block Mined ! 
Nonce : 
2083236893
hash : 
000000000019d6689c085ae165831e934ff763ae46a2a6c172b3f1b60a8ce26f


In [ ]:
def mine_block_num(number):
    
    '''Given a block number it will mine that block.'''
    
    b_version = '{0.08x}'.format(df.loc[number, 'version'])
    if number == 0:
        b_perv_hash = '00'
    else:
        b_perv_hash = df.loc[number - 1, 'hash']
    b_merkel = df.loc[number, 'merkel_root']
    b_time = df.loc[number, 'time']
    b_bit = df.loc[number, 'bits']
    nonce = df.loc[number, 'nonce']
    if nonce > 2e6 :
        near_value = np.random.rdint(nonce - 2e6, nonce - 1e6, dtype='int64')
    else:
        near_value = np.random.randint(1, nonce, dtype='int64')
    
    return mine(b_version, b_perv_hash, b_merkle, b_time, b_bits, near_value)
